In [1]:
import numpy as np
import pandas as pd

from collections import Counter

In [2]:
X = pd.DataFrame([["Sol","Calor","Alta","Debil"],
                ["Sol","Calor","Alta","Fuerte"],
                ["Nublado","Calor","Alta","Debil"],
                ["Lluvia","Templado","Alta","Debil"],
                ["Lluvia","Frio","Normal","Debil"],
                ["Lluvia","Frio","Normal","Fuerte"],
                ["Nublado","Frio","Normal","Fuerte"],
                ["Sol","Templado","Alta","Debil"],
                ["Sol","Frio","Normal","Debil"],
                ["Lluvia","Templado","Normal","Debil"],
                ["Sol","Templado","Normal","Fuerte"],
                ["Nublado","Templado","Alta","Fuerte"],
                ["Nublado","Calor","Normal","Debil"],
                ["Lluvia","Templado","Alta","Fuerte"]],
                columns = ['Cielo', 'Temperatura', 'Humedad', 'Viento'])

y = ['No', 'No', 'Si', 'Si', 'Si', 'No', 'Si', 'No', 'Si', 'Si', 'Si', 'Si', 'Si', 'No']

#display(X)
#display(y)

In [19]:
def construir_arbol(instancias, etiquetas, depth):
    # ALGORITMO RECURSIVO para construcción de un árbol de decisión binario. 
    
    if depth == 0:
        return Hoja(etiquetas)
    
    # Suponemos que estamos parados en la raiz del árbol y tenemos que decidir cómo construirlo. 
    ganancia, pregunta = encontrar_mejor_atributo_y_corte(instancias, etiquetas)
    
    # Criterio de corte: ¿Hay ganancia?
    if ganancia == 0:
        #  Si no hay ganancia en separar, no separamos. 
        return Hoja(etiquetas)
        
    else: 
        # Si hay ganancia en partir el conjunto en 2
        instancias_cumplen, etiquetas_cumplen, instancias_no_cumplen, etiquetas_no_cumplen = partir_segun(pregunta, instancias, etiquetas)
        # partir devuelve instancias y etiquetas que caen en cada rama (izquierda y derecha)
        
        # Paso recursivo (consultar con el computador más cercano)
        sub_arbol_izquierdo = construir_arbol(instancias_cumplen, etiquetas_cumplen,depth-1)
        sub_arbol_derecho   = construir_arbol(instancias_no_cumplen, etiquetas_no_cumplen,depth-1)
        # los pasos anteriores crean todo lo que necesitemos de sub-árbol izquierdo y sub-árbol derecho
        
        # sólo falta conectarlos con un nodo de decisión:
        return Nodo_De_Decision(pregunta, sub_arbol_izquierdo, sub_arbol_derecho)

In [5]:
# Definición de la estructura del árbol. 

class Hoja:
    #  Contiene las cuentas para cada clase (en forma de diccionario)
    #  Por ejemplo, {'Si': 2, 'No': 2}
    def __init__(self, etiquetas):
        self.cuentas = dict(Counter(etiquetas))


class Nodo_De_Decision:
    # Un Nodo de Decisión contiene preguntas y una referencia al sub-árbol izquierdo y al sub-árbol derecho
     
    def __init__(self, pregunta, sub_arbol_izquierdo, sub_arbol_derecho):
        self.pregunta = pregunta
        self.sub_arbol_izquierdo = sub_arbol_izquierdo
        self.sub_arbol_derecho = sub_arbol_derecho
        
        
# Definición de la clase "Pregunta"
class Pregunta:
    def __init__(self, atributo, valor):
        self.atributo = atributo
        self.valor = valor
    
    def cumple(self, instancia):
        # Devuelve verdadero si la instancia cumple con la pregunta
        return instancia[self.atributo] == self.valor
    
    def __repr__(self):
        return "¿Es el valor para {} igual a {}?".format(self.atributo, self.valor)

class Pregunta_num:
    def __init__(self, atributo, valor):
        self.atributo = atributo
        self.valor = valor
    
    def cumple(self, instancia):
        # Devuelve verdadero si la instancia cumple con la pregunta
        return instancia[self.atributo] >= self.valor
    
    def __repr__(self):
        return "¿Es el valor para {} mayor o igual a {}?".format(self.atributo, self.valor)

In [95]:
def entropy(etiquetas):
    import scipy as scp
    
    entropy=0
    tot=float(len(etiquetas)) 
    for i in set(etiquetas):
        val=etiquetas.count(i)
        entropy -= (val/tot)*scp.log2((val/tot))
    return(entropy)

    
def gini(etiquetas):
    
    impureza = 1
    tot=float(len(etiquetas))   

    for i in set(etiquetas):
        val=etiquetas.count(i)
        impureza = impureza - (val/tot)**2
    
    return impureza

def information_gain(et_izq,et_der,method=gini):

    I_par=method(et_izq+et_der)
    I_lft=method(et_izq)
    I_rgt=method(et_der)
    
    t=float(len(et_izq+et_der))
    p_lft=len(et_izq)/t
    p_rgt=len(et_der)/t
     
    ganancia = I_par - (p_lft*I_lft) - (p_rgt*I_rgt)
    
    return(ganancia)
    
def ganancia_gini(etiquetas_rama_izquierda, etiquetas_rama_derecha):
    # COMPLETAR
    
    I_par=gini(etiquetas_rama_izquierda+etiquetas_rama_derecha)
    I_lft=gini(etiquetas_rama_izquierda)
    I_rgt=gini(etiquetas_rama_derecha)
    
    t=float(len(etiquetas_rama_izquierda+etiquetas_rama_derecha))
    p_lft=len(etiquetas_rama_izquierda)/t
    p_rgt=len(etiquetas_rama_derecha)/t
     
    ganancia_gini = I_par - (p_lft*I_lft) - (p_rgt*I_rgt)
    #print(I_par ,p_lft,I_lft,p_rgt,I_rgt)
    
    return ganancia_gini


def partir_segun(pregunta, instancias, etiquetas):
    # Esta función debe separar instancias y etiquetas según si cada instancia cumple o no con la pregunta (ver método 'cumple')
    # COMPLETAR (recomendamos utilizar máscaras para este punto)

    msk=pregunta.cumple(instancias)

    instancias_cumplen = instancias[msk].reset_index(drop=True)
    etiquetas_cumplen = list(pd.Series(etiquetas)[msk])
    instancias_no_cumplen = instancias[~msk].reset_index(drop=True)
    etiquetas_no_cumplen = list(pd.Series(etiquetas)[~msk])

    return instancias_cumplen, etiquetas_cumplen, instancias_no_cumplen, etiquetas_no_cumplen

In [116]:
np.percentile(rn.rand(1000),90)
#type(abse[0])
type(["S","Q"][0])
if type("string") == str:
    print("wii")

wii


In [127]:
type(abse[0])
import numpy
type(abse[0]) in [int,float,numpy.int64]

True

In [140]:
def encontrar_mejor_atributo_y_corte(instancias, etiquetas,method=gini):
    import numpy
    
    max_ganancia = 0
    mejor_pregunta = None
    for columna in instancias.columns:
        if type(instancias[columna][0]) == str:
            
            for valor in set(instancias[columna]):   # Probando corte para atributo y valor 
                pregunta = Pregunta(columna, valor)
                _, et_izq, _, et_der = partir_segun(pregunta, instancias, etiquetas)

                ganancia = information_gain(et_izq, et_der,method)
                if ganancia >= max_ganancia:
                    max_ganancia = ganancia
                    mejor_pregunta = pregunta
            
        elif type(instancias[columna][0]) in [int,float,numpy.int64]:
            
            for i in range(0,100):
                valor=numpy.percentile(instancias[columna],i)
                pregunta = Pregunta_num(columna, valor)
                _, et_izq, _, et_der = partir_segun(pregunta, instancias, etiquetas)                        
        
                ganancia = information_gain(et_izq, et_der,method)
                if ganancia >= max_ganancia:
                    max_ganancia = ganancia
                    mejor_pregunta = pregunta
            
            
            if ganancia >= max_ganancia:
                max_ganancia = ganancia
                mejor_pregunta = pregunta
    
    return(max_ganancia, mejor_pregunta)


def imprimir_arbol(arbol, spacing=""):
    if isinstance(arbol, Hoja):
        print (spacing + "Hoja:", arbol.cuentas)
        return

    print (spacing + str(arbol.pregunta))

    print (spacing + '--> True:')
    imprimir_arbol(arbol.sub_arbol_izquierdo, spacing + "  ")

    print (spacing + '--> False:')
    imprimir_arbol(arbol.sub_arbol_derecho, spacing + "  ")

In [141]:
arbol = construir_arbol(X, y,50)
imprimir_arbol(arbol)

¿Es el valor para Cielo igual a Nublado?
--> True:
('  Hoja:', {'Si': 4})
--> False:
  ¿Es el valor para Humedad igual a Normal?
  --> True:
    ¿Es el valor para Viento igual a Debil?
    --> True:
('      Hoja:', {'Si': 3})
    --> False:
      ¿Es el valor para Temperatura igual a Templado?
      --> True:
('        Hoja:', {'Si': 1})
      --> False:
('        Hoja:', {'No': 1})
  --> False:
    ¿Es el valor para Cielo igual a Sol?
    --> True:
('      Hoja:', {'No': 3})
    --> False:
      ¿Es el valor para Viento igual a Debil?
      --> True:
('        Hoja:', {'Si': 1})
      --> False:
('        Hoja:', {'No': 1})


In [142]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt

abse=rn.randint(0,14,500)

#nX=X.loc[abse].reset_index(drop=True)
cols=['Cielo', 'Temperatura', 'Humedad', 'Viento']
nX=pd.DataFrame([X[i][rn.randint(0,14,500)].reset_index(drop=True) for i in cols]).T #)
ny=list(pd.Series(y)[abse].reset_index(drop=True))

arbol = construir_arbol(nX, ny,50)
imprimir_arbol(arbol)

¿Es el valor para Humedad igual a Normal?
--> True:
  ¿Es el valor para Cielo igual a Lluvia?
  --> True:
    ¿Es el valor para Temperatura igual a Calor?
    --> True:
      ¿Es el valor para Viento igual a Debil?
      --> True:
('        Hoja:', {'Si': 12, 'No': 3})
      --> False:
('        Hoja:', {'Si': 8, 'No': 1})
    --> False:
      ¿Es el valor para Viento igual a Debil?
      --> True:
        ¿Es el valor para Temperatura igual a Templado?
        --> True:
('          Hoja:', {'Si': 13, 'No': 9})
        --> False:
('          Hoja:', {'Si': 7, 'No': 5})
      --> False:
        ¿Es el valor para Temperatura igual a Templado?
        --> True:
('          Hoja:', {'Si': 7, 'No': 7})
        --> False:
('          Hoja:', {'Si': 5, 'No': 4})
  --> False:
    ¿Es el valor para Viento igual a Fuerte?
    --> True:
      ¿Es el valor para Temperatura igual a Calor?
      --> True:
        ¿Es el valor para Cielo igual a Sol?
        --> True:
('          Hoja:', {'Si': 8, 'N

In [24]:
arbol = construir_arbol(X, y,3)
imprimir_arbol(arbol)

¿Es el valor para Cielo igual a Nublado?
--> True:
('  Hoja:', {'Si': 4})
--> False:
  ¿Es el valor para Humedad igual a Normal?
  --> True:
    ¿Es el valor para Viento igual a Debil?
    --> True:
('      Hoja:', {'Si': 3})
    --> False:
('      Hoja:', {'Si': 1, 'No': 1})
  --> False:
    ¿Es el valor para Cielo igual a Sol?
    --> True:
('      Hoja:', {'No': 3})
    --> False:
('      Hoja:', {'Si': 1, 'No': 1})


## Resultado esperado

```
¿Es el valor para Cielo igual a Nublado?
--> True:
  ¿Es el valor para Temperatura igual a Frio?
  --> True:
    Hoja: {'Si': 1}
  --> False:
    ¿Es el valor para Temperatura igual a Templado?
    --> True:
      Hoja: {'Si': 1}
    --> False:
      Hoja: {'Si': 2}
--> False:
  ¿Es el valor para Humedad igual a Normal?
  --> True:
    ¿Es el valor para Viento igual a Fuerte?
    --> True:
      Hoja: {'No': 1, 'Si': 1}
    --> False:
      ¿Es el valor para Cielo igual a Sol?
      --> True:
        Hoja: {'Si': 1}
      --> False:
        Hoja: {'Si': 2}
  --> False:
    ¿Es el valor para Cielo igual a Sol?
    --> True:
      ¿Es el valor para Temperatura igual a Templado?
      --> True:
        Hoja: {'No': 1}
      --> False:
        Hoja: {'No': 2}
    --> False:
      Hoja: {'Si': 1, 'No': 1}
```

## Parte 2 (opcional)
Protocolo sklearn para clasificadores. Completar el protocolo requerido por sklearn. Deben completar la función predict utilizando el árbol para predecir valores de nuevas instancias. 


In [ ]:
def predecir(arbol, x_t):
    # COMPLETAR
    return "Si"
        
class MiClasificadorArbol(): 
    def __init__(self):
        self.arbol = None
        self.columnas = ['Cielo', 'Temperatura', 'Humedad', 'Viento']
    
    def fit(self, X_train, y_train):
        self.arbol = construir_arbol(pd.DataFrame(X_train, columns=self.columnas), y_train)
        return self
    
    def predict(self, X_test):
        predictions = []
        for x_t in X_test:
            x_t_df = pd.DataFrame([x_t], columns=self.columnas).iloc[0]
            prediction = predecir(self.arbol, x_t_df) 
            print(x_t, "predicción ->", prediction)
            predictions.append(prediction)
        return predictions
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        
        accuracy = sum(y_i == y_j for (y_i, y_j) in zip(y_pred, y_test)) / len(y_test)
        return accuracy
        

# Ejemplo de uso
clf = MiClasificadorArbol()

# Tomar en cuenta que sklearn espera numpy arrays:
clf.fit(np.array(X), y)
clf.score(np.array(X), y)
